# Movie reviews Classification

<img src="https://frenzy86.s3.eu-west-2.amazonaws.com/python/nlp/rev.png" width="1200">



In [2]:
import numpy as np
import pandas as pd
import re
from sklearn.datasets import load_files

import warnings
warnings.filterwarnings('ignore')

In [3]:
!wget https://frenzy86.s3.eu-west-2.amazonaws.com/python/nlp/txt_sentoken.zip

--2025-02-19 10:55:11--  https://frenzy86.s3.eu-west-2.amazonaws.com/python/nlp/txt_sentoken.zip
Resolving frenzy86.s3.eu-west-2.amazonaws.com (frenzy86.s3.eu-west-2.amazonaws.com)... 3.5.245.117, 3.5.246.110, 52.95.143.18, ...
Connecting to frenzy86.s3.eu-west-2.amazonaws.com (frenzy86.s3.eu-west-2.amazonaws.com)|3.5.245.117|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2781673 (2.7M) [application/octet-stream]
Saving to: ‘txt_sentoken.zip’

txt_sentoken.zip    100%[===================>]   2.65M  3.94MB/s    in 0.7s    

2025-02-19 10:55:12 (3.94 MB/s) - ‘txt_sentoken.zip’ saved [2781673/2781673]



In [4]:
!unzip txt_sentoken.zip

Archive:  txt_sentoken.zip
   creating: txt_sentoken/
   creating: txt_sentoken/neg/
  inflating: txt_sentoken/neg/cv000_29416.txt  
  inflating: txt_sentoken/neg/cv001_19502.txt  
  inflating: txt_sentoken/neg/cv002_17424.txt  
  inflating: txt_sentoken/neg/cv003_12683.txt  
  inflating: txt_sentoken/neg/cv004_12641.txt  
  inflating: txt_sentoken/neg/cv005_29357.txt  
  inflating: txt_sentoken/neg/cv006_17022.txt  
  inflating: txt_sentoken/neg/cv007_4992.txt  
  inflating: txt_sentoken/neg/cv008_29326.txt  
  inflating: txt_sentoken/neg/cv009_29417.txt  
  inflating: txt_sentoken/neg/cv010_29063.txt  
  inflating: txt_sentoken/neg/cv011_13044.txt  
  inflating: txt_sentoken/neg/cv012_29411.txt  
  inflating: txt_sentoken/neg/cv013_10494.txt  
  inflating: txt_sentoken/neg/cv014_15600.txt  
  inflating: txt_sentoken/neg/cv015_29356.txt  
  inflating: txt_sentoken/neg/cv016_4348.txt  
  inflating: txt_sentoken/neg/cv017_23487.txt  
  inflating: txt_sentoken/neg/cv018_21672.txt  
  inf

In [5]:
movie_data = load_files(r"txt_sentoken")
X_, y = movie_data.data, movie_data.target

In [6]:
df = pd.DataFrame({'text':X_})
df['label'] = y
df

,text,label
0,"b'lisa cholodenko\'s "" high art , "" is an inte...",1
1,"b'wolfgang petersen\'s latest , the perfect st...",0
2,b'i won\x12t even pretend that i have seen the...,0
3,b'there are some works of art that are almost ...,1
4,"b'albert brooks saves the day , just in the ni...",1
...,...,...
1408,"b'hav plenty , as we are told in the beginning...",0
1409,b'brian de palma\'s snake eyes stars nicolas c...,0
1410,b'contact ( pg ) there\'s a moment late in rob...,1
1411,"b""this is a film that i was inclined to like a...",0


In [7]:
## problemi di bytecode
def utfdecode(item):
    cleaned_item = item.decode('utf-8')
    return cleaned_item

df['text'] = df['text'].apply(utfdecode)
df

,text,label
0,"lisa cholodenko's "" high art , "" is an intelli...",1
1,"wolfgang petersen's latest , the perfect storm...",0
2,i wont even pretend that i have seen the othe...,0
3,there are some works of art that are almost im...,1
4,"albert brooks saves the day , just in the nick...",1
...,...,...
1408,"hav plenty , as we are told in the beginning a...",0
1409,brian de palma's snake eyes stars nicolas cage...,0
1410,contact ( pg ) there's a moment late in robert...,1
1411,this is a film that i was inclined to like at ...,0


In [9]:
import re

words_to_remove = ['gargamella', 'doggy', 'word3']
words_pattern = r'\b(?:' + '|'.join(map(re.escape, words_to_remove)) + r')\b'

patterns = {
            r"[a-zA-Z]+n\'t": 'not',
            r'\s+': ' ',             # Substitute multiple spaces with a single space
            r'\d+': '',              # Remove digits
            r'[^\w\s]': '',          # Remove punctuation and symbols
            r'\b\w{1,2}\b': '',      # Remove all tokens less than 2 characters
            r'(http|www)[^\s]+': '', # Remove websites
            words_pattern: ''        # Remove specific words
            }

def clean_column(df, column, patterns):
    df[column] = df[column].str.lower() # Apply lowercase transformation
    for pattern, replacement in patterns.items():
        df[column] = df[column].str.replace(pattern, replacement, regex=True)
    return df

In [13]:
df = clean_column(df,'text',patterns)
df

,text,label
0,lisa cholodenkos high art intelligent qu...,1
1,wolfgang petersens latest the perfect storm ...,0
2,wont even pretend that have seen the other ...,0
3,there are some works art that are almost impo...,1
4,albert brooks saves the day just the nick t...,1
...,...,...
1408,hav plenty are told the beginning and remi...,0
1409,brian palmas snake eyes stars nicolas cages e...,0
1410,contact theres moment late robert zemecki...,1
1411,this film that was inclined like the outs...,0


In [14]:
## 1 Define Features and Target
X = df['text']
y = df['label']

In [15]:
## fare anche WordCloud